In [43]:
import numpy as np
import math
from scipy.stats import norm, gamma, poisson

In [44]:
SeattleA = np.array([[15,12,4],[11,17,4],[0,21,4],[0,0,5]])
SeattleA

array([[15, 12,  4],
       [11, 17,  4],
       [ 0, 21,  4],
       [ 0,  0,  5]])

In [63]:
Tecumseh1 = np.zeros((8,7))
Tecumseh1[0,0:5] = np.array((66 ,87 ,  25  , 22   , 4 ))
Tecumseh1[1,0:5] = np.array((13 ,14 ,  15  ,  9   , 4 ))
Tecumseh1[2,1:6] = np.array((4 ,   4  ,  9  ,  2 ,   1 ))
Tecumseh1[3,2:7] = np.array(( 4   , 3   , 1  ,  1 ,   1))
Tecumseh1[4,3:5] = np.array(( 1,1))
Tecumseh1

array([[66., 87., 25., 22.,  4.,  0.,  0.],
       [13., 14., 15.,  9.,  4.,  0.,  0.],
       [ 0.,  4.,  4.,  9.,  2.,  1.,  0.],
       [ 0.,  0.,  4.,  3.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [45]:
#
# Process data
#

def process(DATA):
    hsA = np.sum(DATA,axis=0)
    isA = np.sum(DATA,axis=1)
    colA = np.array(range(1,len(hsA)+1))
    rowA = np.array(range(0,len(hsA)+1))
    
    xA = np.sum(isA*rowA) # Final size
    N = np.sum(hsA*colA) # Population size
    
    return {'hsA' : hsA, 'isA' : isA, 'colA' : colA, 'rowA' : rowA, 'xA' : xA, 'N' : N}

process(SeattleA)

{'N': 177,
 'colA': array([1, 2, 3]),
 'hsA': array([26, 50, 17]),
 'isA': array([31, 32, 25,  5]),
 'rowA': array([0, 1, 2, 3]),
 'xA': 97}

In [4]:
# Code for setting (local) thresholds in a household of size n.
# 
# Note local thresholds not required for households of size 1.
# Infection rate does not depend upon households size.
#

def thresH(n):
    thres = np.repeat(0.0,n-1)
    thres[0] = np.random.exponential(size=1,scale=1/(n-1))
    if n > 2:
        for i in range(1,n-1):
            thres[i] = thres[i-1] + np.random.exponential(size=1, scale=1/(n-i))
    
    return thres

thresH(5)

array([0.03095242, 0.06987667, 0.52390916, 0.67681226])

In [5]:
def House_epi(n,k,lambda_L):
    if n == 1:
        i = 1
        if k == 0:
            sev = 1
        if k > 0:
            sev = np.random.gamma(k, 1/k, 1)
    
    if n > 1:
        t = thresH(n)
        if k == 0:
            q = np.repeat(1.0,n)
        if k > 0:
            q = np.random.gamma(size=n, shape=k, scale=1/k)
        t = np.append(t, 2*lambda_L*np.sum(q))
        
        i = 0
        test = 0
        while test == 0:
            i += 1
            if t[i-1] > (lambda_L*np.sum(q[0:i])):
                test = 1 
                sev = np.sum(q[0:i])
                
    return np.array([i,sev])

In [6]:
House_epi(1,3,0.21)

array([1.        , 0.76493781])

In [7]:
House_epi(3,5,0.21)

array([1.        , 1.05667602])

In [8]:
#
# Code for simulating a household epidemic using the Sellke construction
#

def House_SEL(hs, lambda_G, lambda_L, k):
    HH = hs.size
    ks = np.array(range(1,HH+1))

    n = np.repeat(ks,hs)

    m = n.size  # Number of households
    N = n.sum() # Population size
    NS = N

    sev = 0
    threshold = 0

    ni = np.repeat(0,m)
    ns = n.copy()

    R = np.random.exponential(size=N)

    while threshold <= lambda_G*sev:
        kk = np.random.choice(a=np.arange(m)+1, size = 1, replace = True, p = ns/(ns.sum()))[0]
        
        hou_epi = House_epi(ns[kk-1],k,lambda_L)
        ns[kk-1] = ns[kk-1] - hou_epi[0]
        sev = sev + hou_epi[1]
        
        ni[kk-1] = n[kk-1]-ns[kk-1] 
        #NS=NS-hou_epi[0]
        NS=ns.sum()
        
        if NS > 0:
            threshold = threshold + np.random.exponential(scale = NS/N, size=1)[0]
        if NS == 0:
            threshold = 2*lambda_G*sev
        
    OUT = np.zeros((HH+1,HH))
    for i in range(1, HH+2):
        for j in range(1,HH+1):
            OUT[i-1,j-1] = sum(n[ni==(i-1)]==j)
    
    return OUT

In [9]:
hs = np.array([1,2,3])
lambda_G = 0.9
lambda_L = 0.21
k=5
House_SEL(hs, lambda_G, lambda_L, 0)

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 2., 0.],
       [0., 0., 3.]])

In [38]:
OUTPUT = np.zeros((run,2))
hs = np.sum(Xdata,axis=0).astype('int')
isB = np.sum(Xdata,axis=1)
rowB = np.arange(hs.size+1)
xB = sum(isB*rowB)
simcount = 0
j = 0

In [39]:
simcount = simcount+1
simcount

1

In [40]:
lambda_G = np.random.exponential(size=1)
lambda_L = np.random.exponential(size=1)
lambda_G, lambda_L

(array([0.44307794]), array([0.69408414]))

In [47]:
J = House_SEL(hs,lambda_G,lambda_L,k)
J

array([[ 79., 104.,  46.,  44.,  12.,   2.,   1.],
       [  0.,   0.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   1.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [48]:
(abs(J-Xdata)).sum()

168.0

In [50]:
isJ = np.sum(J, axis=1)
isJ

array([288.,   1.,   2.,   0.,   0.,   0.,   0.,   0.])

In [60]:
j = j+1
OUTPUT[j-1,] = np.array([lambda_G,lambda_L]).reshape((1,2))

In [61]:
OUTPUT[j-1,]

array([0.44307794, 0.69408414])

In [86]:
#
# Code for running vanilla ABC for households
#
def House_van(Xdata,epsil,k,run):
    """
    Xdata - a data set
    epsil - a numpy array of length 2
    k - int
    run - int
    """
    
    OUTPUT = np.zeros((run,2))
    hs = np.sum(Xdata,axis=0).astype('int')
    isB = np.sum(Xdata,axis=1)
    
    rowB = np.arange(hs.size+1)
    xB = sum(isB*rowB)
    
    simcount = 0
    j = 0
    
    while j<run:
        simcount = simcount+1
        lambda_G = np.random.exponential(size=1)
        lambda_L = np.random.exponential(size=1)
        J = House_SEL(hs,lambda_G,lambda_L,k)
        
        if ((abs(J-Xdata)).sum()) <= epsil[0]:
            isJ = np.sum(J, axis=1)
            if abs(sum(isJ*rowB)-xB) <= epsil[1]:
                j = j+1
                OUTPUT[j-1,] = np.array([lambda_G,lambda_L]).reshape((1,2))
                print(j,simcount)
                
    return {'OUTPUT': OUTPUT, 'simcount' : simcount}
            

In [87]:
Xdata = Tecumseh1
epsil = np.array((100,10))
k = 0
run = 10
House_van(Xdata,epsil,k,run)

1 1281
2 14482
3 14770
4 17734
5 18853
6 19137
7 25107
8 29894
9 43633
10 47846


{'OUTPUT': array([[0.35019624, 0.46158189],
        [0.56228054, 0.32819595],
        [0.43842194, 0.31963222],
        [0.44877964, 0.33719054],
        [0.66813565, 0.16562908],
        [0.57043745, 0.26558129],
        [0.45523261, 0.32304331],
        [0.49612672, 0.31621986],
        [0.59713034, 0.13421639],
        [0.76076266, 0.14317279]]), 'simcount': 47846}

In [83]:
def simSIR(N, beta, gamma):
    # initial number of infectives and susceptibles;
    I = 1
    S = N-1
    
    # recording time;
    t = 0
    times = np.array(0)
    
    # a vector which records the type of event (1=infection, 2=removal)
    type = np.array(1)
    
    while I > 0:
        
        # time to next event;
        t = t + np.random.exponential(size=1,scale= 1/((beta/N)*I*S + gamma*I)) 
        times = np.append(times, t)

        if np.random.uniform(size=1) < beta*S/(beta*S + N*gamma):
            # infection
            I = I+1
            S = S-1
            type = np.append(type,1)
        else:
            #removal
            I = I-1
            type = np.append(type,2)

    return {'removal.times': times[type == 2] - min(times[type == 2]),
           'final.size' : N-S,
           'T' : times[times.size-1] }
    

In [96]:
N=10
beta=0.9
gamma=0.21
out = simSIR(N, beta, gamma)
out

{'T': 17.477588530915444,
 'final.size': 9,
 'removal.times': array([ 0.        ,  1.17030054,  1.37711483,  4.62185701, 10.43306819,
        10.57373736, 10.83514311, 11.44097561, 15.48309699])}

In [97]:
# this is function that simulates an outbreak of a predetermined final size -- to be used for the
# Abakaliki data

def simSIR_constrained(N, beta, gamma, final_size):
    check = 0
    while check == 0:
        out = simSIR(N,beta,gamma)
        if out['final.size'] >= final_size:
            res = out['removal.times']
            check = 1
    
    return res

simSIR_constrained(N, beta, gamma, final_size=5)

array([ 0.        ,  0.62242357,  2.23377393,  6.65353238,  8.25895286,
        9.60161375, 12.59350746, 13.54037791, 17.3008097 , 22.91228597])